## Basic Concepts ##

- Near Realtime (NRT)
- Cluster
- Node (a.k.a a single server)
- Index 
    - a collection of documents that have somewhat similar characterisitcs
- Type
    - a logical category/partition of index
    - mapping types will be REMOVED from 6.0.0
- Document
    - a basic unit of information that can be indexed
- Shards & Replicas
    - index subdivided into multiple pieces --> shards
    - copies of original shards --> replicas
    
### Source ###
1. https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started-concepts.html

## Common CMDs ##

### Check Cluster Health ###
`_cat`
    - /_cat/health?v
    - /_cat/nodes?v

### API Pattern ###
```<HTTP Verb> /<Index>/<Endpoint>/<ID>```
    
### List All Indices ###
/_cat/indices?v

### Create an Index ###
Create an index "customer" and list all indices

```shell
$ curl -X PUT "localhost:9200/customer?pretty"                                                                           
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "customer"
}
```

```shell
$ curl -X GET "localhost:9200/_cat/indices?v"                                                                            
health status index    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   customer Nx9Xd9owRyyTYqRUnDsP_Q   1   1          0            0       230b           230b
```

### Index and Query a Document ###
**Index a Dcouement**
```shell
$ curl -X PUT "localhost:9200/customer/_doc/1?pretty" -H 'Content-Type: application/json' -d'                            
{
  "name": "John Snow"
}
'
```
**response:**
```shell
{
  "_index" : "customer",
  "_type" : "_doc",
  "_id" : "1",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 0,
  "_primary_term" : 1
}
```

**Query a document**
```shell
$ curl -X GET "localhost:9200/customer/_doc/1?pretty"
```
**response:**
```shell
{
  "_index" : "customer",
  "_type" : "_doc",
  "_id" : "1",
  "_version" : 1,
  "_seq_no" : 0,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "name" : "John Snow"
  }
}
```

### Delete an Index ###
```shell
$ curl -X DELETE "localhost:9200/customer?pretty"                                                                        
curl -X GET "localhost:9200/_cat/indices?v"

{
  "acknowledged" : true
}
health status index uuid pri rep docs.count docs.deleted store.size pri.store.size
```

### Modifying Data ###

**Replacing Documents**
- `PUT` data with same `index` and same `id`

**If no id is specify while indexing a document, a random id is generated by ES**
    - But in this case, `POST` keyword should be use
```shell
$ curl -X POST "localhost:9200/customer/_doc?pretty" -H 'Content-Type: application/json' -d'                             
{
  "name": "Jane Doe"
}
'
```

### Updating Documents ###
```shell
curl -X POST "localhost:9200/customer/_update/1?pretty" -H 'Content-Type: application/json' -d'
{
  "doc": { "name": "Jane Doe" }
}
'
```

```shell
curl -X POST "localhost:9200/customer/_update/1?pretty" -H 'Content-Type: application/json' -d'
{
  "doc": { "name": "Jane Doe", "age": 20 }
}
'
```

```shell
curl -X POST "localhost:9200/customer/_update/1?pretty" -H 'Content-Type: application/json' -d'
{
  "script" : "ctx._source.age += 5"
}
'
```

### Deleting Documents ###
`DELETE /customer/_doc/2?pretty`

### Batch Processing ###
Batch processing is NOT atomic.

```shell
curl -X POST "localhost:9200/customer/_bulk?pretty" -H 'Content-Type: application/json' -d'
{"index":{"_id":"1"}}
{"name": "John Doe" }
{"index":{"_id":"2"}}
{"name": "Jane Doe" }
'
```
```shell
curl -X POST "localhost:9200/customer/_bulk?pretty" -H 'Content-Type: application/json' -d'
{"update":{"_id":"1"}}
{"doc": { "name": "John Doe becomes Jane Doe" } }
{"delete":{"_id":"2"}}
'
```